In [90]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from dotenv import load_dotenv

In [91]:
load_dotenv()

costos = pd.read_csv(os.getenv('RUTA_COSTOS'))
insumos = pd.read_csv(os.getenv('RUTA_INSUMOS'))
proveedores = pd.read_csv(os.getenv('RUTA_PROVEEDORES_PROCESADOS'))
arriendos = pd.read_csv(os.getenv('RUTA_ARRIENDOS'))

In [92]:
costos.head(5)

,Boleta,Fecha,Proveedor,Producto,Cantidad,Precio_Unitario,Dscto,Total,Año,Mes,Trimestre,MesNombre,Semana,Dia,DiaAño,DiaSemana
0,0,2023-03-02,76.746.599-8,1.0,1.0,2390.00,0.0,2390.0,2023,3,1,Mar,9,2,61,Thursday
1,0,2023-03-02,76.746.599-8,2.0,1.0,4100.00,0.0,4100.0,2023,3,1,Mar,9,2,61,Thursday
2,1,2023-03-02,76.979.850-1,7.0,20.0,138.79,0.0,2775.8,2023,3,1,Mar,9,2,61,Thursday
3,2,2023-03-02,77.318.793-2,3.0,50.0,195.00,0.0,9750.0,2023,3,1,Mar,9,2,61,Thursday
4,3,2023-03-02,77.372.135-1,56.0,1.0,7900.00,0.0,7900.0,2023,3,1,Mar,9,2,61,Thursday


In [93]:
costos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550 entries, 0 to 2549
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Boleta           2550 non-null   int64  
 1   Fecha            2550 non-null   object 
 2   Proveedor        2550 non-null   object 
 3   Producto         2550 non-null   float64
 4   Cantidad         2545 non-null   float64
 5   Precio_Unitario  2544 non-null   float64
 6   Dscto            2550 non-null   float64
 7   Total            2550 non-null   float64
 8   Año              2550 non-null   int64  
 9   Mes              2550 non-null   int64  
 10  Trimestre        2550 non-null   int64  
 11  MesNombre        2550 non-null   object 
 12  Semana           2550 non-null   int64  
 13  Dia              2550 non-null   int64  
 14  DiaAño           2550 non-null   int64  
 15  DiaSemana        2550 non-null   object 
dtypes: float64(5), int64(7), object(4)
memory usage: 318.9+ KB


In [94]:
costos.describe()

,Boleta,Producto,Cantidad,Precio_Unitario,Dscto,Total,Año,Mes,Trimestre,Semana,Dia,DiaAño
count,2550.000000,2550.000000,2545.000000,2544.000000,2550.000000,2550.000000,2550.000000,2550.000000,2550.000000,2550.000000,2550.000000,2550.000000
mean,585.183137,356.755294,5.141269,5544.634726,558.831373,6181.811241,2023.002353,7.968627,2.961176,32.838431,16.787843,227.801961
std,334.998300,454.469640,15.636859,36579.658819,15409.747013,32984.663683,0.048460,2.795887,0.935159,12.282560,8.802667,86.027929
min,0.000000,-1.000000,0.060000,6.700000,0.000000,0.000000,2023.000000,1.000000,1.000000,2.000000,1.000000,10.000000
25%,303.000000,27.000000,1.000000,1000.000000,0.000000,1547.000000,2023.000000,6.000000,2.000000,22.000000,10.000000,153.000000
50%,583.500000,56.000000,1.000000,1890.000000,0.000000,2249.000000,2023.000000,8.000000,3.000000,33.000000,17.000000,231.000000
75%,863.000000,1008.000000,2.054000,3282.500000,0.000000,4239.061500,2023.000000,10.000000,4.000000,43.000000,24.000000,299.000000
max,1167.000000,1053.000000,200.000000,580000.000000,560100.000000,580000.000000,2024.000000,12.000000,4.000000,52.000000,31.000000,365.000000


In [99]:
# Estimar viajes
df = costos.merge(proveedores, 
                  how='inner', 
                  left_on='Proveedor', 
                  right_on='ID_PROVEEDOR').merge(insumos, 
                                                 how='inner', 
                                                 left_on='Producto',
                                                 right_on='ID_PRODUCTO')
df = df[['Boleta','Fecha','NOMBRE_x','Producto','Total','SECTOR','TIPO']]

In [114]:
fechas = df[df['Producto']==1004]['Fecha'].to_list()
sectores_sin_pago = ['PEÑUELAS', 'LA CANTERA', 'DELIVERY']
df1 = df[~(df['Fecha'].isin(fechas)) & (df['TIPO']!='SERVICIOS BÁSICOS') & ~(df['SECTOR'].isin(sectores_sin_pago))]
df1 = df1.groupby(by=['Fecha','SECTOR']).agg({'Boleta':'nunique',
                                              'NOMBRE_x':'nunique',
                                              'Producto':'nunique',
                                              'Total':'sum',
                                              'TIPO':'nunique'}).reset_index()

In [118]:
fechas_agregar = df1['Fecha'].to_list()
fechas_agregar

['2023-03-03',
 '2023-03-22',
 '2023-05-08',
 '2023-05-09',
 '2023-05-12',
 '2023-06-24',
 '2023-06-26',
 '2023-06-27',
 '2023-07-04',
 '2023-07-07',
 '2023-07-10',
 '2023-07-12',
 '2023-07-13',
 '2023-07-14',
 '2023-07-15',
 '2023-07-17',
 '2023-07-18',
 '2023-07-19',
 '2023-07-20',
 '2023-07-26',
 '2023-07-27',
 '2023-07-31',
 '2023-07-31',
 '2023-08-02',
 '2023-08-03',
 '2023-08-05',
 '2023-08-09',
 '2023-08-10',
 '2023-08-11',
 '2023-08-11',
 '2023-08-14',
 '2023-08-16',
 '2023-08-18',
 '2023-08-19',
 '2023-08-24',
 '2023-08-30',
 '2023-08-31',
 '2023-09-06',
 '2023-09-07',
 '2023-09-12',
 '2023-09-13',
 '2023-09-14',
 '2023-09-15',
 '2023-09-15',
 '2023-09-16',
 '2023-09-22',
 '2023-09-23',
 '2023-09-25',
 '2023-09-29',
 '2023-09-30',
 '2023-10-03',
 '2023-10-06',
 '2023-10-17',
 '2023-10-18',
 '2023-10-19',
 '2023-10-23',
 '2023-10-31',
 '2023-11-02',
 '2023-11-03',
 '2023-11-06',
 '2023-11-10',
 '2023-11-15',
 '2023-11-16',
 '2023-11-23',
 '2023-11-30',
 '2023-12-01',
 '2023-12-

In [125]:
df1.head(50)

,Fecha,SECTOR,Boleta,NOMBRE_x,Producto,Total,TIPO
0,2023-03-03,COQUIMBO,1,1,1,5371.000,1
1,2023-03-22,LA GARZA,1,1,1,18100.000,1
2,2023-05-08,LA SERENA,1,1,2,6424.560,1
3,2023-05-09,COQUIMBO,4,4,9,25850.766,3
4,2023-05-12,COQUIMBO,7,7,14,35454.682,4
5,2023-06-24,COQUIMBO,1,1,1,2000.000,1
6,2023-06-26,COQUIMBO,1,1,1,2766.000,1
7,2023-06-27,COQUIMBO,1,1,1,2670.000,1
8,2023-07-04,COQUIMBO,1,1,7,28067.000,3
9,2023-07-07,COQUIMBO,12,12,39,102523.000,6


In [128]:
df1.tail(25)

,Fecha,SECTOR,Boleta,NOMBRE_x,Producto,Total,TIPO
50,2023-10-03,COQUIMBO,2,2,5,12395.0,2
51,2023-10-06,COQUIMBO,8,8,20,50766.0,5
52,2023-10-17,LA GARZA,1,1,1,14500.0,1
53,2023-10-18,COQUIMBO,12,12,16,79909.0,5
54,2023-10-19,COQUIMBO,3,3,14,45561.0,5
55,2023-10-23,REGIMIENTO ARICA,1,1,23,95493.0,3
56,2023-10-31,COQUIMBO,1,1,18,85595.0,2
57,2023-11-02,COQUIMBO,8,8,16,62106.0,5
58,2023-11-03,COQUIMBO,4,4,5,17843.0,3
59,2023-11-06,COQUIMBO,1,1,1,9677.0,1


In [131]:
for i in range(len(df)):
    print(df1.loc[i, 'SECTOR'])

COQUIMBO
LA GARZA
LA SERENA
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
REGIMIENTO ARICA
COQUIMBO
LA GARZA
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
LA SERENA
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
LA GARZA
COQUIMBO
LA GARZA
LA SERENA
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
LA GARZA
COQUIMBO
COQUIMBO
REGIMIENTO ARICA
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
LA SERENA
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
COQUIMBO
LA SERENA
LA GARZA


KeyError: 75